<a href="https://colab.research.google.com/github/EddyDavies/Trade_With_Twitter/blob/main/ipynb/Transformer_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Check GPU

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

zsh:1: command not found: nvidia-smi


Check TPU

Check Memory

In [3]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 17.2 gigabytes of available RAM

Not using a high-RAM runtime


In [4]:
# !git clone https://github.com/EddyDavies/Trade_With_Twitter.git

In [5]:
!ls .//

Transformer_Inference.ipynb     text_extraction_with_bert.ipynb


In [6]:
%cd Trade_With_Twitter/src
!ls

[Errno 2] No such file or directory: 'Trade_With_Twitter/src'
/Users/edwarddavies/Git/Trade_with_Twitter/ipynb
Transformer_Inference.ipynb     text_extraction_with_bert.ipynb


In [7]:
# from google.colab import drive
#
# drive.mount('/content/drive')

In [8]:
!pip install -r ../requirements.txt

You should consider upgrading via the '/Users/edwarddavies/Git/Trade_with_Twitter/venv/bin/python -m pip install --upgrade pip' command.


In [9]:
import os
import pandas as pd
from tqdm import tqdm

from utils import get_date_array, get_month_array
from sentiment.inference import to_dict_of_lists, get_paths

source, results_folder = get_paths()
# source, results_folder = get_paths(data_folder='../../drive/MyDrive/ColabData')

In [10]:
dates_range = ("2017-01-01", "2021-06-20")
months_range = ["Jan 17", "Jun 21"]

months = get_month_array(months_range)
dates = get_date_array(dates_range)

In [11]:
from transformers import pipeline

sentiment_analysis = pipeline("sentiment-analysis")

In [17]:
from sentiment.inference import get_tweets, get_sentiments, save_sentiments, check_last_day

for date in dates:
    ids, tweets = get_tweets(date, source)

    percentage_per_chunk = 10
    save_every = 2000
    results = get_sentiments(date, tweets, sentiment_analysis, 
        save_every=save_every, percentage_per_chunk=percentage_per_chunk)

    outputs = to_dict_of_lists(results)
    outputs["ids"] = ids
    save_sentiments(outputs, date)

    check_last_day(results, results_folder, date)

KeyboardInterrupt: 

In [ ]:
!ls ../

In [ ]:
# !rm -r ../data/bitcoin_scores

In [ ]:
print(results_folder)

In [ ]:
!git pull